### 훈련데이터 및 JSON 파일 업데이트
* Excel파일 수정 후 사용

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts

In [2]:
answers= pd.read_excel('코로나 표준질문 통합 Nov..xlsx',sheet_name='New표준질문 답변')
data = pd.read_excel('코로나 표준질문 통합 Nov..xlsx',sheet_name='별도질문')

In [3]:
def processing(a):
    a = a.dropna(subset=[a.columns[1]])
    temp = []
    for i in range(len(a.iloc[:,0])):
        if i == 0:
            temp.append(a.iloc[:,0].iloc[0])
        else:
            if str(a.iloc[:,0].iloc[i])=='nan':
                temp.append(temp[-1])
            else : temp.append(a.iloc[:,0].iloc[i])
    a.iloc[:,0] = temp
    data2 = a
    return data2

In [4]:
newdata = pd.DataFrame()
for i in np.arange(0,14,2):
    d = processing(data.iloc[:,1+i:3+i])
    d.columns = ['코드','질문']
    newdata = pd.concat([newdata,d],axis=0)
newdata = newdata.reset_index(drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [5]:
label = dict()
j=0
for i in np.unique(newdata['코드']):
    label[i] = j
    j +=1

In [6]:
conv = answers[answers['주제']=='대화']['코드'].to_list()

In [7]:
with open('conv.txt','w') as f:
    f.write('\n'.join(conv))

In [8]:
import json
with open('codetolab.json','w') as fp:
    json.dump(label,fp)

#### JSON update

In [9]:
#Nested dict
labtoinfo = dict()
for i in label:
    temp = dict()
    temp['code'] = i
    temp['ques'] = answers[answers['코드'] == i]['질문'].iloc[0]
    temp['anstype'] = answers[answers['코드'] == i]['답변타입'].iloc[0]
    temp['txtans'] = answers[answers['코드'] == i]['답변'].iloc[0]
    temp['imgans'] = answers[answers['코드'] == i]['이미지'].iloc[0]
    temp['ref'] = answers[answers['코드'] == i]['출처'].iloc[0]
    labtoinfo[str(label[i])] = temp

In [ ]:
import json
with open('labtoinfo.json','w') as fp:
    json.dump(labtoinfo,fp)

#### Training data update

In [11]:
temp = []
for i in newdata['코드']:
    temp.append(label[i])
newdata['label']=pd.Series(temp)

In [12]:
X_train, X_test, y_train, y_test = tts(newdata[['코드','질문']],newdata['label'],test_size=.3)

In [13]:
X_val, X_test, y_val, y_test = tts(X_test,y_test, test_size=.33)

In [14]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [15]:
class_train = pd.concat([X_train,y_train],axis=1)
class_val = pd.concat([X_val,y_val],axis=1)
class_test = pd.concat([X_test,y_test],axis=1)

#### CSV 파일 생성

In [ ]:
class_train.to_csv('class_train.csv',encoding='cp949')
class_val.to_csv('class_val.csv',encoding='cp949')
class_test.to_csv('class_test.csv',encoding='cp949')

#### 텍스트 파일 생성

In [18]:
class_train.to_csv('text_class_train.txt',encoding='cp949',index=False, sep='\t')
class_val.to_csv('C:text_class_val.txt',encoding='cp949',index=False, sep='\t')
class_test.to_csv('C:text_class_test.txt',encoding='cp949',index=False, sep='\t')